In [1]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime, timedelta
import pytz

In [2]:
# Initialize MT5 connection
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()

In [3]:
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M1

In [111]:
#Get the time ranges in utc
timezone = pytz.timezone('UTC')
start_time = datetime.now(timezone) - timedelta(days=7) + timedelta(hours=3)
end_time = datetime.now(timezone) - timedelta(minutes=2) + timedelta(hours=3)

rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)

# Create a DataFrame
ohlc_df = pd.DataFrame(rates)
ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
ohlc_df.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
ohlc_df = ohlc_df.reindex(labels=["time","open", "high", "low", "close"], axis = 1)

,time,open,high,low,close
0,2024-08-02 12:01:00,1.08169,1.08172,1.08151,1.08165
1,2024-08-02 12:02:00,1.08165,1.08172,1.08163,1.08171
2,2024-08-02 12:03:00,1.08172,1.08181,1.08171,1.08179
3,2024-08-02 12:04:00,1.08177,1.08179,1.08165,1.08173
4,2024-08-02 12:05:00,1.08175,1.08175,1.08164,1.08171
...,...,...,...,...,...
7192,2024-08-09 11:54:00,1.09133,1.09140,1.09131,1.09139
7193,2024-08-09 11:55:00,1.09139,1.09144,1.09135,1.09143
7194,2024-08-09 11:56:00,1.09143,1.09148,1.09138,1.09146
7195,2024-08-09 11:57:00,1.09144,1.09155,1.09143,1.09153


In [ ]:
#function to fetch the latest 1 min ohlc data
def fetch_latest_data():
    now_local = datetime.now(local_tz)
    now_mt5 = now_local.astimezone(mt5_tz)
    end_time = now_mt5 - timedelta( minutes=1)
    start_time = now_mt5 - timedelta( minutes=2)

    rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)
    one_df = pd.DataFrame(rates)
    one_df = pd.DataFrame(rates)
    one_df['time'] = pd.to_datetime(one_df['time'], unit='s')
    one_df['time'] = one_df['time'].dt.tz_localize('UTC').dt.tz_convert(mt5_tz)
    one_df.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    one_df = one_df.reindex(labels=["time","open", "high", "low", "close"], axis = 1)

    
    # df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)
    # target_timezone = pytz.timezone('Etc/GMT-3')
    # df['time'] = df['time'].dt.tz_convert(target_timezone)
    # df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S%z')
    # # df.set_index('time', inplace=True)
    # df.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    # df = df.reindex(labels=["time","open", "high", "low", "close"], axis = 1)
    return one_df

In [ ]:
# Infinite loop to fetch data every minute
try:
    while True:
        new_data = fetch_latest_data()
        ohlc_df = pd.concat([ohlc_df,new_data],ignore_index=True)
        ohlc_df.drop_duplicates(subset="time", keep="last", ignore_index= True)
        print(ohlc_df)
        # Wait for 60 seconds before fetching new data
        time.sleep(60)
except KeyboardInterrupt:
    print("Process interrupted by user")

# Shutdown MT5 connection
mt5.shutdown()
